In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
import json



In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
import json

# --------------------------
# 1. Setup directories
# --------------------------
RAW_DIR = "staging/raw/api/"
CSV_FILE = "staging/processed/covid_pakistan.csv"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(os.path.dirname(CSV_FILE), exist_ok=True)


In [ ]:
# --------------------------
# 2. Define date range
# --------------------------
start_date = datetime(2020, 1, 1)
end_date = datetime(2022, 12, 31)

In [ ]:

# --------------------------
# 3. Prepare storage
# --------------------------
all_records = []   # For CSV


In [ ]:
# --------------------------
# 4. Loop over dates
# --------------------------
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    url = f"https://covid-api.com/api/reports?date={date_str}&iso=PAK&region_name=Pakistan"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Save raw JSON (one file per date)
        raw_file = os.path.join(RAW_DIR, f"pakistan_{date_str}.json")
        with open(raw_file, "w") as f:
            json.dump(data, f, indent=2)

        # Normalize JSON → Flatten for CSV
        if "data" in data and data["data"]:
            df = pd.json_normalize(data["data"])
            df["report_date"] = date_str  # add date column
            all_records.append(df)

        print(f"Fetched data for {date_str}")

    except Exception as e:
        print(f"❌ Failed for {date_str}: {e}")

    current_date += timedelta(days=1)


Fetched data for 2020-01-01
Fetched data for 2020-01-02
Fetched data for 2020-01-03
Fetched data for 2020-01-04
Fetched data for 2020-01-05
Fetched data for 2020-01-06
Fetched data for 2020-01-07
Fetched data for 2020-01-08
Fetched data for 2020-01-09
Fetched data for 2020-01-10
Fetched data for 2020-01-11
Fetched data for 2020-01-12
Fetched data for 2020-01-13
Fetched data for 2020-01-14
Fetched data for 2020-01-15
Fetched data for 2020-01-16
Fetched data for 2020-01-17
Fetched data for 2020-01-18
Fetched data for 2020-01-19
Fetched data for 2020-01-20
Fetched data for 2020-01-21
Fetched data for 2020-01-22
Fetched data for 2020-01-23
Fetched data for 2020-01-24
Fetched data for 2020-01-25
Fetched data for 2020-01-26
Fetched data for 2020-01-27
Fetched data for 2020-01-28
Fetched data for 2020-01-29
Fetched data for 2020-01-30
Fetched data for 2020-01-31
Fetched data for 2020-02-01
Fetched data for 2020-02-02
Fetched data for 2020-02-03
Fetched data for 2020-02-04
Fetched data for 202

In [ ]:
# --------------------------
# 5. Save CSV
# --------------------------
if all_records:
    final_df = pd.concat(all_records, ignore_index=True)
    final_df.to_csv(CSV_FILE, index=False)
    print(f"\n✅ Data saved to {CSV_FILE}")
else:
    print("\n⚠️ No data fetched!")



✅ Data saved to staging/processed/covid_pakistan.csv


In [ ]:
# Run in a Colab code cell
!pip install boto3 tqdm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # follow the prompts
# Example CSV path: /content/drive/MyDrive/covid_pakistan.csv


Mounted at /content/drive


In [ ]:
import os
import getpass
import boto3
import botocore

# Enter these when prompted (they are hidden)
aws_access_key = getpass.getpass("")
aws_secret_key = getpass.getpass("")
aws_region = input("AWS Region (eu-north-1) ")
bucket_name = input("S3 bucket name (s3://dataingestionandstoring/raw/): ")

# Optional: set environment vars for this runtime (not persisted)
os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key
os.environ['AWS_DEFAULT_REGION'] = aws_region

# Create a session + client
session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=aws_region
)
s3 = session.client('s3')
print("S3 client ready for bucket:", bucket_name)


In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError

# AWS credentials (replace with your keys)
AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
REGION = "eu-north-1"
BUCKET_NAME = "dataingestionandstoring"

# Initialize boto3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=REGION
)

# File paths
local_file = "staging/processed/covid_pakistan.csv"
s3_file = "raw/covid_pakistan.csv"

# Upload
try:
    s3.upload_file(local_file, BUCKET_NAME, s3_file)
    print("✅ Upload Successful!")
except FileNotFoundError:
    print("❌ The file was not found.")
except NoCredentialsError:
    print("❌ Credentials not available.")


✅ Upload Successful!
